# 회귀 기반 추천 시스템

In [14]:
#!pip3 install pandas

In [15]:
import pandas as pd

In [16]:
travel_df = pd.read_csv('./data/travel.csv')


In [17]:
travel_df[['AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM']] = travel_df[['AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM']].astype(int)

In [18]:
# !pip3 install scikit-learn

In [19]:
from sklearn.model_selection import train_test_split

X = travel_df.drop('DGSTFN', axis=1)
y = travel_df['DGSTFN']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [20]:
#!pip3 install catboost

In [21]:
from catboost import Pool
import catboost
print(catboost.__version__)

cat_features = [
    'GENDER', 'AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 
    'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 
    'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM', 
    'VISIT_AREA_NM', 'MVMN_NM']

X_train_pool = Pool(X_train, y_train, cat_features=cat_features)
X_train_pool = Pool(X_test, y_test, cat_features=cat_features)


1.2.7


In [22]:
!pip3 catboost --version

ERROR: unknown command "catboost"


In [23]:
from catboost import CatBoostRegressor

cb_reg = CatBoostRegressor(
    n_estimators=100,      # 반복횟수 (내부 예측기 개수)
    depth=5,               # 개별 트리의 최대 깊이
    learning_rate=0.03,    
    loss_function='RMSE',  # 손실함수 (기본값)
    eval_metric='RMSE'      # 평가 지표 (기본값)
    )

cb_reg.fit(X_train_pool, eval_set=X_test, verbose=10)

CatBoostError: Invalid type of 'eval_set': <class 'pandas.core.frame.DataFrame'>, while expected Pool or (X, y) or filename, or list thereof.

In [ ]:
col_importance = pd.DataFrame({
    'column': X_train.columns,
    'importance' : cb_reg.feature_importances_    
    
})

##### 추천 시스템 구축

1. 방문지 목록을 생성
2. 사용자 특성 입력
3. 가상 만족도 예측
4. 만족도가 높은 순으로 추천

In [ ]:
visit_areas = travel_df['VISIT_AREA_NM'].unique()
visit_areas.shape
visit_areas[:10]

In [ ]:
    # 'GENDER', 'AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 
    # 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 
    # 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM', 
    # 'VISIT_AREA_NM', 'MVMN_NM'
user_input = ['여', 60, 4, 4, 4, 4, 4, 4, 4, 4, 1, 2, '방문지', '자가용']
pred_results = []

for area in visit_areas:
    user_input[-2] = area
    dgstfn_pred = cb_reg.predict(user_input)
    pred_results.append(dgstfn_pred)

pred_results[:10]

In [ ]:
result_df = pd.DataFrame({
    'VISIT_AREA_NM': visit_areas,
    'DGSTFN_PRED': pred_results    
})

result_df.sort_values(by='DGSTFN_PRED', ascending=False).head(10)